In [ ]:
from pattern import web
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

url = ["http://www.aishub.net/stations?page=" + str(i) for i in range(1, 16)]

def getTable(html):
    """
    Function
    --------
    Scans html for data table and returns as a dataframe.
    
    Parameters
    ----------
    html : URL address of data table
    
    Returns
    -------
    Full dataframe
    """
    
    #Nested functions
    def extractfromi(element):
        if '<i' in str(element):
            return web.Element(element)('i')[0].attrs['title']
        else:
            return element
        
    def extractfromspan(element):
        if '<span' in str(element):
            span = web.Element(element)('span')[0]
            if span.content == '': #Get tile
                return span.attrs['title']
            else:
                return span.content
        else:
            element
        
    def extractfroma(element):
        if '<a ' in str(element):
            a = web.Element(element)('a')[0]
            if a.content == '': #Get tile
                try:
                    return a.attrs['title']
                except:
                    return None
            else:
                return a.content
        else:
            return element
        
    
    #Extract all data from webpage
    u = web.URL(html)
    code = u.download(cached=True)
    dom = web.Element(code)
    
    #get list of headers of data columns
    table_header = dom('tr')[0]
    headers = [table_header('th a')[i].content for i in range(len(table_header('th a')))]
    headers.append('Source')
    
    #Define the result as a dataframe containing all named columns
    result = pd.DataFrame({k: [] for k in headers})
    
    #Get table data and append to dataframe
    table_data = dom('tbody tr')
    for i in range(len(table_data)):
        row = table_data[i]
        tds = row('td')
        cells = [tds[j].content for j in range(len(tds))]
        
        #Make table readable for user
        for k in range(len(cells)):
            if "<i " in str(cells[k]):
                cells[k] = extractfromi(cells[k])
            elif "<span" in str(cells[k]):
                cells[k] = extractfromspan(cells[k])
            elif "<a " in str(cells[k]):
                cells[k] = extractfroma(cells[k])
        
        result = result.append(pd.Series(cells, index=headers), ignore_index=True)
    
    return result

station_list = pd.DataFrame()
for u in url:
    station_list = station_list.append(getTable(u))
station_list = station_list.set_index('ID')

station_list

In [ ]:
import geopandas as gpd

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.rename(columns={'name': 'Country'})

world = world.set_index('Country')
world.plot()


In [ ]:
def mapByDistinct(stations):
    """
    Function
    --------
    Maps each country by how many of its distint ships are active
    
    Parameters
    ----------
    stations: dataframe of each station on record
    
    Returns
    -------
    Nothing
    """
    stations = stations[(stations.Status == "Online") & (stations.Country != 'Unknown')]
    stations.Country = stations.Country.replace("Viet Nam", "Vietnam").replace("Saint Martin (FR)", "France").replace("Taiwan (Province of China)", "Taiwan").replace("Russian Federation", "Russia")
    stations = stations.set_index(['Country','Source'])
    distinct_total = stations.Distinct.astype(int).sum(level="Country")
    
    #append to world
    global world
    world["Ship_circulation_est"] = distinct_total
    
    #plot result
    result = world.plot(column='Ship_circulation_est', 
                        legend=True,
                        legend_kwds={'label': 'Active ships per country',
                                     'orientation': 'horizontal'},
                        missing_kwds={'color':'lightgrey'})
    plt.savefig('shipsPerCountry.png')
    
    return result

ships_per_Country = mapByDistinct(station_list)

In [ ]:
#Now less see the amount of active ships per 1000000 persons for each country
world["Ships_per_1e6"] = (world.Ship_circulation_est / world.pop_est) * 1e6


ships_per_1e6 = world.plot(column='Ships_per_1e6', 
                           legend=True,
                           legend_kwds={'label': 'Active ships relative to million persons',
                                        'orientation': 'horizontal'},
                           missing_kwds={'color': 'lightgray'})
plt.savefig('shipsPer1e6.png')


In [ ]:
from scipy.special import erf
def GaussianProb():
    """
    Function
    --------
    Works out gaussian porobability that ship belongs to specific demographic. Adds to dataframe
    
    Parameters
    ----------
    None
    
    Returns
    -------
    None
    """
    cdf = lambda x, mu, sig: 0.5 * (1 + erf((x - mu) / (sig * np.sqrt(2))))
    
    global world
    world["CDF"] = cdf(world.Ship_circulation_est, world.Ship_circulation_est.mean(), world.Ship_circulation_est.std())
    
    #If NaN assume zero
    world.CDF = world.CDF.fillna(float(0))
    
GaussianProb()

In [ ]:
fig, ax = plt.subplots(figsize = (20, 20))
ships_cdf = world.plot(column='CDF', 
                           legend=True,
                           legend_kwds={'label': 'Likleyhood active ship belongs to country',
                                        'orientation': 'horizontal'},
                       ax=ax)
plt.savefig('shipsCDF.png')

In [ ]:
ports = pd.DataFrame({'Country': station_list.Country, 'Location': station_list.Location}).drop_duplicates()

In [ ]:
#We will have to find a way of adding geometric loction to each station
#First less add a column to ports labeled geometry and set each entry to None
ports["Longitude"] = [None for i in range(ports.shape[0])]
ports["Latitude"] = [None for i in range(ports.shape[0])]
cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
ports

In [ ]:
import wikipedia as wiki
def getGeometry(station):
    """
    Function
    --------
    Gets geometric location for each port, by mining wikipedia.
    
    Parameters
    ----------
    station: Entry in dataframe ports
    
    Returns
    -------
    geometry: estimated location in float form
    """
    coords = None
    i = 0
    engine = wiki.search(station.Location + ', ' + station.Country)
    while i < len(engine):
        try:
            page = wiki.page(engine[i])
            coords = page.coordinates
            break
        except wiki.exceptions.DisambiguationError as dmbError:
            engine = wiki.search(station.Location + ', ' + station.Country + ', port')
        except:
            coords = None
        i = i + 1
    
    try:
        station.Longitude = float(coords[0])
        station.Latitude = float(coords[1])
    except:
        station.Longitude = float(np.nan)
        station.Latitude = float(np.nan)
        
    return station

j = 0
while j < 586:
    ports.iloc[j, :] = getGeometry(ports.iloc[j, :])
    j = j + 1

In [ ]:
ports

In [ ]:
#Drop nan items from Dataframe
ports = ports.dropna()
ports

In [ ]:
#Convert pors to geoDataFrame
geoPorts = gpd.GeoDataFrame(ports, geometry=gpd.points_from_xy(ports.Latitude, ports.Longitude))

In [ ]:
fig, ax = plt.subplots(figsize = (20, 20))
world.plot(column='CDF', 
           legend=True,
           legend_kwds={'label': 'Likleyhood active ship belongs to country',
                        'orientation': 'horizontal'},
           ax=ax)
geoPorts.plot(ax=ax)

In [ ]:
def nearestStation(start, dest, location):
    """
    Function
    --------
    Function will take a coordinate pair, start 
    country and destination country, then return the nearest 
    station. Assume this is in the case of a lost and 
    stranded ship that is in need of finding its way back. We 
    will try to find this through checking the 2 norm.
    
    Parameters
    ----------
    start: Country the ship left
    dest: Country the ship wants to get to
    location: Coordinates of ship
    
    Returns
    -------
    Station: Location the ship needs to go to to get back on 
    course.
    """
    
    global world, geoPorts
    origin = world[world.index == start].geometry.bounds
    end = world[world.index == dest].geometry.bounds
    
    #Assume longest route to get vector direction in unit form
    vector = {'x': end.maxx[0] - origin.minx[0], 'y': end.maxy[0] - origin.miny[0]}
    vector = {'x': vector['x'] / np.sqrt(pow(vector['x'], 2) + pow(vector['y'], 2)), 
          'y': vector['y'] / np.sqrt(pow(vector['x'], 2) + pow(vector['y'], 2))}
    
    locV = {'x': location[0], 'y': location[0]}
    
    station = geoPorts.iloc[0, :]
    #First try and find if there are any stations between location and destination
    possibleStations = geoPorts[(geoPorts.Longitude > min(end.maxx[0], location[0])) & 
                                (geoPorts.Longitude < max(end.maxx[0], location[0])) &
                                (geoPorts.Latitude > min(end.maxy[0], location[0])) &
                                (geoPorts.Latitude < max(end.maxy[0], location[0]))]
    #If empty check any towards start point
    if possibleStations.shape[0] == 0:
        possibleStations = geoPorts[(geoPorts.Longitude > min(origin.maxx[0], location[0])) & 
                                    (geoPorts.Longitude < max(origin.maxx[0], location[0])) &
                                    (geoPorts.Latitude > min(origin.maxy[0], location[0])) &
                                    (geoPorts.Latitude < max(origin.maxy[0], location[0]))]
        if possibleStations.shape[0] == 0:
            possibleStations = geoPorts
            
    #Now get distance between each station and location
    possibleStations["distance"] = np.power(np.power(np.abs(possibleStations.Longitude - location[0]), 2) + 
                                           np.power(np.abs(possibleStations.Latitude - location[1]), 2), 1/2)
    
    ps = pd.DataFrame(possibleStations)
    
    #find entry with minimum distance and return
    station = ps[ps.distance == ps.distance.min()]
    
    script = "Please head towards " + str(station.Location[0]) + ", " + str(station.Country[0]) + ".\n In direction: Longitude = " + str(station.Longitude[0] - locV['x']) + ", Latitude = " + str(station.Latitude[0] - locV['y'])
    
    return station, script

In [ ]:
nearestStation('Australia', 'Angola', (-50, 75))

In [ ]:
#Add a colour scheme to which one can recognise the status of each station
color = []
for s in geoPorts.Location:
    status = (station_list[station_list.Location == s]).Status.values[0]
    uptime = float(((station_list[station_list.Location == s]).Uptime.values[0]).strip('%')) / 100
    
    c = None
    if status == 'Online':
        c = (0, 1, uptime)
    elif status == 'Offline > 10 minutes':
        c = (1, 115/256, uptime)
    elif status == 'Offline > 24 hours':
        c = (1, 0, uptime)
    elif status == "Dead":
        c = (0, 0, uptime)
        
    color.append(c)

geoPorts["Status"] = color

In [ ]:
geoPorts[geoPorts.Status != (0, 1, 1)]

In [ ]:
fig, ax = plt.subplots(figsize = (20, 20))
world.plot(column='CDF', 
           legend=True,
           legend_kwds={'label': 'Likleyhood active ship belongs to country',
                        'orientation': 'horizontal'},
           ax=ax)
geoPorts[geoPorts.Status != (0, 1, 1)].plot(color=geoPorts[geoPorts.Status != (0, 1, 1)]["Status"], legend=["Offline, Uptime < 100%"], ax=ax)

In [ ]:
def nearestStationInOperation(start, dest, location):
    """
    Function
    --------
    Function will take a coordinate pair, start 
    country and destination country, then return the nearest 
    station. Assume this is in the case of a lost and 
    stranded ship that is in need of finding its way back. We 
    will try to find this through checking the 2 norm. This
    time we will exclude stations that are offline, below 100%
    uptime or dead.
    
    Parameters
    ----------
    start: Country the ship left
    dest: Country the ship wants to get to
    location: Coordinates of ship
    
    Returns
    -------
    Station: Location the ship needs to go to to get back on 
    course.
    """
    
    global world, geoPorts
    origin = world[world.index == start].geometry.bounds
    end = world[world.index == dest].geometry.bounds
    
    #Assume longest route to get vector direction in unit form
    vector = {'x': end.maxx[0] - origin.minx[0], 'y': end.maxy[0] - origin.miny[0]}
    vector = {'x': vector['x'] / np.sqrt(pow(vector['x'], 2) + pow(vector['y'], 2)), 
          'y': vector['y'] / np.sqrt(pow(vector['x'], 2) + pow(vector['y'], 2))}
    
    locV = {'x': location[0], 'y': location[0]}
    
    station = geoPorts[geoPorts.Status == (0, 1.0, 1.0)].iloc[0, :]
    #First try and find if there are any stations between location and destination
    possibleStations = geoPorts[(geoPorts.Longitude > min(end.maxx[0], location[0])) & 
                                (geoPorts.Longitude < max(end.maxx[0], location[0])) &
                                (geoPorts.Latitude > min(end.maxy[0], location[0])) &
                                (geoPorts.Latitude < max(end.maxy[0], location[0])) &
                                (geoPorts.Status == (0, 1.0, 1.0))]
    #If empty check any towards start point
    if possibleStations.shape[0] == 0:
        possibleStations = geoPorts[(geoPorts.Longitude > min(origin.maxx[0], location[0])) & 
                                    (geoPorts.Longitude < max(origin.maxx[0], location[0])) &
                                    (geoPorts.Latitude > min(origin.maxy[0], location[0])) &
                                    (geoPorts.Latitude < max(origin.maxy[0], location[0])) &
                                    (geoPorts.Status == (0, 1.0, 1.0))]
        if possibleStations.shape[0] == 0:
            possibleStations = geoPorts[geoPorts.Status == (0, 1.0, 1.0)]
            
    #Now get distance between each station and location
    possibleStations["distance"] = np.power(np.power(np.abs(possibleStations.Longitude - location[0]), 2) + 
                                           np.power(np.abs(possibleStations.Latitude - location[1]), 2), 1/2)
    
    ps = pd.DataFrame(possibleStations)
    
    #find entry with minimum distance and return
    station = ps[ps.distance == ps.distance.min()]
    
    script = "Please head towards " + str(station.Location[0]) + ", " + str(station.Country[0]) + ".\n In direction: Longitude = " + str(station.Longitude[0] - locV['x']) + ", Latitude = " + str(station.Latitude[0] - locV['y'])
    
    return station, script

In [ ]:
nearestStationInOperation('Australia', 'Angola', (-50, 75))

In [ ]:
world[world.continent == 'Europe'].Ship_circulation_est.sum()

In [ ]:
#So less create a function that takes a series of rules
#based on known amount of ships in certain regions of 
#the world and predicts how many ships are in each 
#countries waters

#Expand geomtry and add as pairs of intervals to a new column called bounds
Bounds = world.geometry.bounds
world["bounds"] = [([Bounds.minx.iloc[c], Bounds.maxx.iloc[c]], [Bounds.miny.iloc[c], Bounds.maxy.iloc[c]]) for c in range(Bounds.shape[0])]
   
#Example rule function
def knowShipsNorthernHemisphere(N):
    global world
    north = world[world.bounds.minx >= 0]
    north.Ship_circulation_est = [0 for c in range(north.shape[0])]
    xsum = lambda x_: (N <= (north.CDF * x_).sum())
    return xsum

knowShipsNorthernHemisphere(50000)

In [ ]:
from scipy.optimize import linprog
def predictShipsPerCountry(Conditions, constraints):
    """
    Function
    --------
    Calculates optimal prediction of ships per a country 
    using a set of conditions and constraints
    
    Parameters
    ----------
    Conditions: Set of conditions without contraints
                as matrix
                
    contraints: Vector of constraints
    
    Returns
    -------
    x_: array containing predicted ships for each country
    """
    global world
    c_ = -1 * world.CDF.values #Assume maxization probems require the inverse of the minimise problem
    x_ = linprog(c_,
                 A_ub=Conditions,
                 b_ub=constraints,
                 options={'maxiter': 100, 'disp': False, 'autoscale': False, 'presolve': False, 'rr': False})
    return x_

In [ ]:
#Now create a set of conditions and allocate a value for each one

#Assume that if each country has equal weight, 1, thae total ships in circulation is at most 100000
a0 = [-1 for i in range(world.shape[0])]
b0 = -100000

#Assume for every 1e9 $, a country can chose to have one ship
a1 = [-1 * world.gdp_md_est.iloc[i] / 1e3 for i in range(world.shape[0])]
b1 = -1 * world.shape[0]

#Assume for a every 1e6 people, a country must have one ship available
a2 = [world.pop_est.iloc[i] / 1e6 for i in range(world.shape[0])]
b2 = world.shape[0]

#Assume knowledge of at least a certain number of ships around each continent
#Europe
a3 = [(-1 if world.continent.iloc[i] == 'Europe' else 1) for i in range(world.shape[0])]
b3 = world.Ship_circulation_est.sum() - world.Ship_circulation_est[world.continent == 'Europe'].sum()

#North America
a4 = [(-1 if world.continent.iloc[i] == 'North America' else 1) for i in range(world.shape[0])]
b4 = world.Ship_circulation_est.sum() - world.Ship_circulation_est[world.continent == 'North America'].sum()

#South America
a5 = [(-1 if world.continent.iloc[i] == 'South America' else 1) for i in range(world.shape[0])]
b5 = world.Ship_circulation_est.sum() - world.Ship_circulation_est[world.continent == 'South America'].sum()

#Asia
a6 = [(-1 if world.continent.iloc[i] == 'Asia' else 1) for i in range(world.shape[0])]
b6 = world.Ship_circulation_est.sum() - world.Ship_circulation_est[world.continent == 'Asia'].sum()

#Oceania
a7 = [(-1 if world.continent.iloc[i] == 'Oceania' else 1) for i in range(world.shape[0])]
b7 = world.Ship_circulation_est.sum() - world.Ship_circulation_est[world.continent == 'Oceania'].sum()

#Africa
a8 = [(-1 if world.continent.iloc[i] == 'Africa' else 1) for i in range(world.shape[0])]
b8 = world.Ship_circulation_est.sum() - world.Ship_circulation_est[world.continent == 'Africa'].sum()

#Antarctica
a9 = [(-1 if world.continent.iloc[i] == 'Antarctica' else 1) for i in range(world.shape[0])]
b9 = world.Ship_circulation_est.sum() - world.Ship_circulation_est[world.continent == 'Antarctica'].sum()

#Now put these rules into processible form for calculating the prediction
A = [a0, a1, a2, a3, a4, a5, a6, a7, a8, a9]
b_ = [b0, b1, b2, b3, b4, b5, b6, b7, b8, b9]

#There are at least 39 rules missing to ensure we get a number for each value, less generate randomly
#for i in range(39):
#    a = np.random.choice(np.linspace(-5, 5), size=world.shape[0])
#    b = np.random.choice(np.linspace(50000, 500000))
#    A.append(A)
#    b_.append(b)

In [ ]:
#Now calculate a prediction from the above rules
x_ = predictShipsPerCountry(A, b_)

In [ ]:
x_